In [3]:
# Installation de la bibliothèque ClearML
!pip -q install clearml
# Librairies classiques
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [25]:
# initialisation ClearML
!clearml-init

# Initialisation d'une Task et mise en place du Logger
from clearml import Task, Logger
task = Task.init(project_name="Tumor Classification", task_name="third model")
logger = Logger.current_logger()

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7ce8440e9300>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 445, in cb
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/usr/local/bin/clearml-init", line 5, in <module>
    from clearml.cli.config.__main__ import main
  File "/usr/local/lib/python3.12/dist-packages/clearml/__init__.py", line 5, in <module>
    from .task import Task
  File "/usr/local/lib/python3.12/dist-packages/clearml/task.py", line 76, in <module>
    from .binding.frameworks.fastai_bind import PatchFastai
  File "/usr/local/lib/python3.12/dist-packages/clearml/binding/frameworks/fastai_bind.py", line 202, in <module>
    class PatchFastaiV2(object):
  File "/usr/local/lib/python3.12/dist-packages/clearml/binding/frameworks/fastai_bind.py", line 227, in PatchFastaiV2
    from fastai.learner import Recorder
  File "/usr/local/lib/python3.12/dist-packages/fastai/learner.py", line 7, in <module>
    from 

In [ ]:
# Chemins vers les dossiers train/test
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.listdir("/content/drive/MyDrive/dataset_brain tumor"))

train_data="/content/drive/MyDrive/dataset_brain tumor/Training"
test_data="/content/drive/MyDrive/brain tumor/Testing"

# Paramètres
IMG_SIZE = (224, 224)
BATCH_SIZE = 64

# Chargement en dataset TensorFlow
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Mounted at /content/drive
['Testing', 'Training']
Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [8]:
# Convertir les datasets en numpy arrays
X_train, y_train = [], []
for images, labels in train_ds:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

X_test, y_test = [], []
for images, labels in test_ds:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())
X_test = np.concatenate(X_test)
y_test = np.concatenate(y_test)

# Normalisation
X_train = X_train / 255.0
X_test = X_test / 255.0

# Encodage one-hot des labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

class_names = train_ds.class_names


In [9]:
# Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=5,
    zoom_range=0.02,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False
)
data_gen.fit(X_train)


In [10]:
# Modèle CNN
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(512, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))  # nombre de classes


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
# Compilation
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# Callbacks (EarlyStopping + ReduceLROnPlateau)
early_stop = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1
)


In [13]:
# Entraînement (flow + fit)
train_generator = data_gen.flow(
    X_train, y_train,
    batch_size=32,
    shuffle=True
)

history = model.fit(
    train_generator,
    epochs=30,  # EarlyStopping peut arrêter avant
    validation_data=(X_test, y_test),
    callbacks=[early_stop, reduce_lr]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



Epoch 1/30
71/90 ━━━━━━━━━━━━━━━━━━━━ 1:38 5s/step - accuracy: 0.5555 - loss: 1.1343

72/90 ━━━━━━━━━━━━━━━━━━━━ 1:33 5s/step - accuracy: 0.5559 - loss: 1.1336

75/90 ━━━━━━━━━━━━━━━━━━━━ 1:17 5s/step - accuracy: 0.5571 - loss: 1.1314

80/90 ━━━━━━━━━━━━━━━━━━━━ 51s 5s/step - accuracy: 0.5591 - loss: 1.1282

90/90 ━━━━━━━━━━━━━━━━━━━━ 497s 5s/step - accuracy: 0.5632 - loss: 1.1220 - val_accuracy: 0.2563 - val_loss: 2.7062 - learning_rate: 0.0010
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 483s 5s/step - accuracy: 0.6545 - loss: 0.9024 - val_accuracy: 0.3731 - val_loss: 2.2155 - learning_rate: 0.0010
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 490s 5s/step - accuracy: 0.6538 - loss: 0.8934 - val_accuracy: 0.4061 - val_loss: 2.0178 - learning_rate: 0.0010
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 488s 5s/step - accuracy: 0.7307 - loss: 0.7352 - val_accuracy: 0.2563 - val_loss: 6.8340 - learning_rate: 0.0010
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 496s 5s/step - accuracy: 0.7000 - loss: 0.7522 - val_accuracy: 0.3883 - val_loss: 2.7460 - learning_rate: 0.0010
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7442 - loss: 0.6814
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
90/90 ━━━━━━━━━━━━━━━━━━━━ 480s 5s/step - accuracy: 0.7443 - loss: 0.6810 - val_accuracy: 0.3553

84/90 ━━━━━━━━━━━━━━━━━━━━ 30s 5s/step - accuracy: 0.8663 - loss: 0.4140

88/90 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - accuracy: 0.8657 - loss: 0.4148

90/90 ━━━━━━━━━━━━━━━━━━━━ 477s 5s/step - accuracy: 0.8653 - loss: 0.4153 - val_accuracy: 0.4619 - val_loss: 1.5203 - learning_rate: 2.0000e-04
Epoch 11/30


90/90 ━━━━━━━━━━━━━━━━━━━━ 481s 5s/step - accuracy: 0.8732 - loss: 0.3863 - val_accuracy: 0.5888 - val_loss: 1.2379 - learning_rate: 2.0000e-04
Epoch 12/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 485s 5s/step - accuracy: 0.8693 - loss: 0.3618 - val_accuracy: 0.6396 - val_loss: 1.4286 - learning_rate: 2.0000e-04
Epoch 13/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 504s 5s/step - accuracy: 0.8648 - loss: 0.3806 - val_accuracy: 0.5787 - val_loss: 1.5264 - learning_rate: 2.0000e-04
Epoch 14/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8743 - loss: 0.3642
Epoch 14: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
90/90 ━━━━━━━━━━━━━━━━━━━━ 485s 5s/step - accuracy: 0.8743 - loss: 0.3640 - val_accuracy: 0.5355 - val_loss: 1.2724 - learning_rate: 2.0000e-04
Epoch 15/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 484s 5s/step - accuracy: 0.8993 - loss: 0.3082 - val_accuracy: 0.5964 - val_loss: 1.4241 - learning_rate: 4.0000e-05
Epoch 16/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 486s 5s/step - accuracy: 0.9043 - loss: 0.3

In [21]:
# Récupération des courbes depuis l'historique d'entraînement
train_loss = history.history.get("loss", [])
val_loss = history.history.get("val_loss", [])
train_acc = history.history.get("accuracy", [])
val_acc = history.history.get("val_accuracy", [])

# Envoi des courbes à ClearML
for epoch in range(len(train_loss)):
    Logger.current_logger().report_scalar("loss", "train", train_loss[epoch], epoch)
    Logger.current_logger().report_scalar("loss", "val", val_loss[epoch], epoch)
    Logger.current_logger().report_scalar("accuracy", "train", train_acc[epoch], epoch)
    Logger.current_logger().report_scalar("accuracy", "val", val_acc[epoch], epoch)

In [14]:
# Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6528 - loss: 1.2780
Test Loss: 1.2796140909194946
Test Accuracy: 0.6472080945968628


In [22]:
# Envoi à ClearML
Logger.current_logger().report_scalar("accuracy", "test", test_acc, len(train_loss))
Logger.current_logger().report_scalar("loss", "test", test_loss, len(train_loss))

In [24]:
# Matrice de confusion
y_true = np.argmax(y_test, axis=1)  # convertir y_test one-hot en labels
y_pred = np.argmax(model.predict(X_test), axis=1)

cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
plt.title("Matrice de confusion")
plt.xlabel("Prédit")
plt.ylabel("Réel")

# 5️⃣ Envoi à ClearML
Logger.current_logger().report_matplotlib_figure(
    title="Confusion Matrix",
    series="evaluation",
    iteration=len(train_loss),
    figure=fig
)
plt.close(fig)
print("✅ Matrice de confusion envoyée à ClearML.")

13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step
✅ Matrice de confusion envoyée à ClearML.
